- **Collect:** University of San Diego
- **Course:** Natural Language Processing and GenAI (AAI-520)
- **Final Project:** Financial Analysis Aystem Powered by Agentic AI
- **Professor:** Kahila Mokhtari Jadid
- **Team Members:** Pros Loung, Dennis Arpurayil, Divya Kamath 
- **GitHub Repository:** (https://github.com/ploung/AAI_520_NLP_Final_Project.git)



# 1. Project Introduction

The project builds a real-world financial analysis system powered by agentic AI. That can reason, plan, and act, coordinating multiple specialized LLM agents to handle complex financial tasks end-to-end. Agentic AI moves beyond scripted flows: it routes tasks, critiques itself, and improves iteratively. The multi-agent stacks parse news, earnings, and market signals at scale.

# 2. Set the enviroment

In [11]:
# Uncomment and run the following lines to install necessary packages
#!pip install yfinance
#!pip install openai
#!pip install langchain 
#!pip install pandas 
#!pip install numpy 
#!pip install requests
#!pip install langchain-community

# 3. Import libraries

In [12]:
import yfinance as yf 
import requests
#import pandas as pd
#import matplotlib.pyplot as plt
#from datetime import datetime, timedelta

# 4. Design Financial Data Retrieval Agent

In [13]:
# Define stock list to fetch data for and analyze
stock_list = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "META"]

# Create a function to fetch stock data
def fetch_stock_data(symbol):
    stock = yf.Ticker(symbol)
    return stock.history(period="1mo")

# Example usage
stock_result = list(map(fetch_stock_data, stock_list)) # Fetch data for all stocks in the list

# Display stock history data
for stock, stock_data in zip(stock_list, stock_result):
    print(stock, stock_data.head(), "\n")  # Print the first few rows of each stock's data


AAPL                                  Open        High         Low       Close  \
Date                                                                        
2025-08-25 00:00:00-04:00  226.479996  229.300003  226.229996  227.160004   
2025-08-26 00:00:00-04:00  226.869995  229.490005  224.690002  229.309998   
2025-08-27 00:00:00-04:00  228.610001  230.899994  228.259995  230.490005   
2025-08-28 00:00:00-04:00  230.820007  233.410004  229.339996  232.559998   
2025-08-29 00:00:00-04:00  232.509995  233.380005  231.369995  232.139999   

                             Volume  Dividends  Stock Splits  
Date                                                          
2025-08-25 00:00:00-04:00  30983100        0.0           0.0  
2025-08-26 00:00:00-04:00  54575100        0.0           0.0  
2025-08-27 00:00:00-04:00  31259500        0.0           0.0  
2025-08-28 00:00:00-04:00  38074700        0.0           0.0  
2025-08-29 00:00:00-04:00  39418400        0.0           0.0   

MSFT        

# 5. Design News Retrieval and Processing Agent

In [14]:
import requests

def get_news(symbol, api_key):
    url = f"https://newsapi.org/v2/everything?q={symbol}&apiKey={api_key}"
    response = requests.get(url)
    articles = response.json()["articles"]
    return [article["content"] for article in articles]

def preprocess_news(news_list):
    return [news.lower().replace("\n", " ") for news in news_list]

In [15]:
stock_news_dict = {}
api_key = "529848f53e0f4767b2a82dbd288dd370"  # Replace with your NewsAPI key

for symbol in stock_list:
    raw_news = get_news(symbol, api_key)
    cleaned_news = preprocess_news(raw_news)
    stock_news_dict[symbol] = cleaned_news

# Display the news articles for a specific stock symbol
for symbol, news_items in stock_news_dict.items():
    print(f"\nNews for {symbol}:")
    for i, item in enumerate(news_items[:3]):  # Show first 3 articles
        print(f"{i+1}. {item[:100]}...")  # Preview first 100 characters


News for AAPL:
1. let's turn now to our stock of the day. apple's new iphone 17 and iphone air are officially availabl...
 first up, markets are looking to start the week in the gr...
3. yahoo finance's john hyland tracks tuesday's top moving stocks and biggest market stories in this ma...

News for MSFT:
1. big data refers to a vast and diverse collection of structured, unstructured and semi-structured dat...
2. d-wave quantum qbts and rigetti computing rgti are stepping into the spotlight as speculative pure p...
3. shares of voice ai technology company soundhound ai (nasdaq:soun) jumped 7% in the afternoon session...

News for GOOGL:
1. 08 september 2025, bavaria, munich: the google brand logo (logo, symbol, emblem) can be seen at the ...
2. ionq (ionq) is a pioneering quantum computing company driving the industry forward with its advanced...
3. the year 2025 has been tricky for google parent alphabet  (googl) and its patient investors, but the...

News for AMZN:
1. ankara, turkiy

# 6. Prompt Chaining

In [16]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
openai_llm = OpenAI(openai_api_key="YOUR_API_KEY")  # Replace with your actual OpenAI API key

template = PromptTemplate(input_variables=["text"], template="Classify this financial news: {text}")
# chain = LLMChain(prompt=template, llm=your_llm)
chain = LLMChain(prompt=template, llm=openai_llm)  # Replace with actual LLM instance

def classify_news(news_list):
    return [chain.run(text=news) for news in news_list]

#classified_news = classify_news(preprocess_news(get_news("AAPL", "YOUR_NEWSAPI_KEY")))  # Replace with your actual NewsAPI key

# 7. Routing Logic Agent

In [17]:
def route_content(classification):
    if "earnings" in classification:
        return "Earnings Analyzer"
    elif "market" in classification:
        return "Market Analyzer"
    else:
        return "General News Analyzer"

# 7. Evaluator Optimizer

In [18]:
def evaluate_output(output):
    score = len(output)  # Placeholder for actual scoring logic
    return score

def optimize_output(output, score):
    if score < 100:
        return output + " [Refined]"
    return output

# 8. Memory Across Runs

In [19]:
memory_log = []

def store_memory(note):
    memory_log.append(note)

def retrieve_memory():
    return memory_log[-3:]  # Last 3 notes

## 9. Evaluation & Iteration
- Example of system critiquing itself and re-running an agent  
- Show before/after outputs  

## 10. Conclusion
- What worked well  
- Challenges  
- Future improvements (better data sources, advanced models, memory persistence)  